In [11]:
import fastf1 as ff1
import csv
import pandas as pd
import os
import time

In [12]:
cache_dir = 'F1_data/cache'
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

ff1.Cache.enable_cache('F1_data/cache')

In [16]:
from concurrent.futures import ThreadPoolExecutor
import fastf1 as ff1
import pandas as pd

years = range(2022, 2023)
sessions = ['Q', 'R']

for year in years:
    yearly_weather_data = []

    # Loop through each round in the event schedule
    for round_number in range(1, ff1.get_event_schedule(year)['RoundNumber'].max() + 1):
        for session_type in sessions:
            with ThreadPoolExecutor(max_workers=20) as executor:
                try:
                    # Load the session data with laps
                    session = ff1.get_session(year, round_number, session_type)
                    session.load(laps=True)  # Load laps and other data

                    # Retrieve weather data
                    weather_data = session.weather_data.copy()

                    # Associate each weather entry with the corresponding lap numbers for all drivers
                    # Merge based on nearest time to each lap
                    lap_data = session.laps[['Driver', 'LapNumber', 'Time']]
                    weather_data['LapNumber'] = None
                    weather_data['Driver'] = None

                    for driver in lap_data['Driver'].unique():
                        driver_laps = lap_data[lap_data['Driver'] == driver]
                        driver_weather = weather_data.copy()
                        driver_weather['Driver'] = driver

                        # Find the nearest lap time for each weather timestamp
                        driver_weather['LapNumber'] = driver_weather['Time'].apply(
                            lambda time: driver_laps.iloc[(driver_laps['Time'] - time).abs().argsort()[:1]]['LapNumber'].values[0]
                            if not driver_laps.empty else None
                        )

                        # Append driver's weather data to yearly data
                        yearly_weather_data.append(driver_weather)

                    print(f"Collected weather data for {session_type} {year} Round {round_number}")
                except Exception as e:
                    print(f"Error loading data for {session_type} {year} Round {round_number}: {e}")

    # Combine all collected weather data for the year into a single DataFrame
    yearly_weather_df = pd.concat(yearly_weather_data, ignore_index=True)

    # Save the data to a CSV file for that year
    output_file = f'weather_data_with_lap_numbers_{year}.csv'
    yearly_weather_df.to_csv(output_file, index=False)
    print(f"Data for {year} saved to {output_file}")


logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	No cached data found for season_schedule. Loading data...
_api           INFO 	Fetching season schedule...
logger      WARNING 	Failed to load schedule from F1 API backend!
logger      WARNING 	Failed to load schedule from Ergast API backend!


ValueError: Failed to load any schedule data.